In [7]:
import os

import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import torchaudio

# torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np
import pandas as pd
import torchvision

def bucket(x, count):
    if count==10:
        return x
    if count==5:
        return (x+1)//2
    if count==3:
        return min((x+2)//3, 3)
    if count==2:
        return (x+4)//5
    if count==1:
        return 0 if x==0 else 1

def fill(meldir, buckets, dir):
    X_test = None
    y_test = []
    yc_test = []
    for l in open(meldir + '.csv'):
        label = int(l[-2])
        if(l[-3:-1] == '10'):
            id = (l[len('/home/swli2/slowfast/data/video'):-len('_final.mp4 xx\n')])
            label = 10
        else:
            id = (l[len('/home/swli2/slowfast/data/video'):-len('_final.mp4 x\n')])
        # print(label, meldir)
        

        if(label == 0):
            filename = dir+'/'+meldir+'/'+(id)+'n.pt'
            potential = torch.load(filename)
            # potential = torch.unsqueeze(potential, axis=0)
            if(X_test == None):
                X_test = potential
            else:
                X_test = torch.cat((X_test, potential), axis=0)
            
            y_test.append(0)
            yc_test.append(0)
        else:
            filename = dir + '/'+meldir+'/'+(id)+'y.pt'
            potential = torch.load(filename)
            # potential = torch.unsqueeze(potential, axis=0)
            if(X_test == None):
                X_test = potential
            else:
                X_test = torch.cat((X_test, potential), axis=0)
            y_test.append(1)
            yc_test.append(bucket(label, buckets))
        # print(X_test[-1].shape, y_test[-1], yc_test[-1])
        # print(filename)
    return (X_test), torch.Tensor(y_test), torch.Tensor(yc_test)

buckets=1
dtu = 'mae_filtered'
X_test, y_test, y_mc_test = fill('test', buckets, dtu)
print("finished test")
X_train, y_train, y_mc_train = fill('train', buckets, dtu)


X_train = torch.mean(torch.reshape(X_train, (72, 5, 768)), axis=1)
X_test = torch.mean(torch.reshape(X_test, (175, 5, 768)), axis=1)

finished test


In [8]:
print(X_test.shape, X_train.shape)

torch.Size([175, 768]) torch.Size([72, 768])


In [14]:
import torch.nn as nn
import torch
from torchmetrics.classification import BinaryAUROC, AveragePrecision

class lin(nn.Module):
    def __init__(self):
        super().__init__()
        self.ReLU = nn.ReLU()
        self.lin1 = nn.Linear(768, 1024)
        self.lin2 = nn.Linear(1024, buckets+1)
        self.lina = nn.Linear(768, buckets+1)
        # self.bn2 = nn.BatchNorm1d(16)
        self.bn = nn.BatchNorm1d(buckets+1)
        

    def forward(self, x):
        res = self.lin2((self.ReLU(self.lin1(x))))
        # res = self.lina(x)
        # res = self.bn(res)
        return res


t1accs = []
mses = []
f1s = []
aurocs = []
mmses = []
baurocs = []
bauprcs = []
bf1s = []
baccs = []
brecalls = []
num_iters = 200
for _ in range(15):
    mt = 'lin'
    model = lin()

    model.train()
    model.to('cuda')
    optim = torch.optim.Adam(model.parameters(), lr=3e-4)


    input = torch.Tensor(X_train).to('cuda')
    if(dtu == 'mae_full' and mt != 'GRU'):
        input = input.swapaxes(1, 2)

    target = torch.Tensor(y_mc_train).type(torch.LongTensor).to('cuda')

    print(input.shape, target.shape)

    weights = torch.Tensor([1, (36/8), (36/5), (36/6), (36/12), (36/5)]).to('cuda')
    weights = torch.Tensor([1, 36/30]).to('cuda')

    def loss_func(pred, target, l=1):
        targets = torch.nn.functional.one_hot(target, pred.shape[1])
        targets = torch.mul(targets, weights)
        spred = torch.clip(nn.Softmax(dim=1)(pred), 1e-20, 1-1e-20)
        wce = -torch.mean(targets*torch.log(spred))

        # mce = nn.CrossEntropyLoss()(pred, target)

        t = (target!=0).type(torch.float32)
        bce = nn.BCELoss()(1-spred[:, 0], t)

        mpred = torch.sum((torch.softmax(pred, axis=1) * torch.Tensor(np.arange(buckets+1)).to('cuda')), axis=1)
        mse = torch.mean((mpred - target)**2)
        return 1*wce + 0*l*bce + 0*mse

    for i in range(num_iters):
        model.to('cuda')
        model.train()
        ind = np.random.randint(0, 72, 64)
        if(dtu == 'mae_full'):
            pred = model(input[ind, :, :])
        else:
            pred = model(input[ind, :])

        loss = loss_func(pred, target[ind])
        optim.zero_grad()
        loss.backward()
        optim.step()

        if(i%num_iters == num_iters-1):
            print(loss.item())
            with torch.no_grad():
                model.eval()
                model.to('cpu')
                inp = torch.Tensor(X_test).to('cpu')
                if(dtu == 'mae_full' and mt != 'GRU'):
                    inp = inp.swapaxes(1, 2)
                y = torch.Tensor(y_mc_test).to('cpu')

                pred = model(inp)#.reshape(y.shape)
                s = torch.softmax(pred, axis=1)
                pred = torch.argmax(pred, axis=1)
                # pred = torch.softmax(pred, axis=1)
                # pred = torch.round(torch.sum((pred * torch.Tensor([0, 1, 2, 3, 4, 5])), axis=1))
                print("Test MSE:", nn.MSELoss()(pred, y))
                tmse = nn.MSELoss()(pred, y)
                # pred = torch.round(pred)
                # print(pred)
                # print(y)

                acc = torch.mean((pred==y).type(torch.float32))
                print("Test ACC:", acc)
                print()
                t1macro = 0
                f1macro = 0
                aurocmacro = 0
                msemacro = 0
                for b in range(buckets+1):
                    precision = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((pred==b).type(torch.float32))
                    recall = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((y==b).type(torch.float32))
                    f1 = (2*precision*recall)/(precision+recall)
                    f1 = torch.nan_to_num(f1, 0, 0, 0)
                    auroc = BinaryAUROC()(s[:, b] , (y==b).type(torch.float32))
                    mse = torch.sum((y==b)*(pred - y)**2)/torch.sum((y==b).type(torch.float32))
                    print("Class: " + str(b), "Precision: " + str(precision.item()), "Recall: " + str(recall.item()), "F1: " + str(f1.item()), "AUROC: " + str(auroc.item()), "MSE: " + str(mse.item()))
                    t1macro+=recall
                    f1macro+= f1
                    aurocmacro+=auroc
                    msemacro+=mse
                
                t = (y!=0).type(torch.float32)

                print("Metrics of Interest (MSE, f1, auroc, mse, Bin AUROC): \nTest MSE: " + str(tmse.item()) + " F1: " 
            + str(f1macro.item()/(buckets+1))+ ", AUROC: " + str(aurocmacro.item()/(buckets+1))+ ", MSE: " 
            + str(msemacro.item()/(buckets+1)) + ", Bin AUROC: " + str(BinaryAUROC()(1-s[:, 0], t).item()) + "\n")
                
                t1accs.append(acc.item())
                mses.append(tmse.item())
                f1s.append(f1macro.item()/(buckets+1))
                aurocs.append(aurocmacro.item()/(buckets+1))
                mmses.append(msemacro.item()/(buckets+1))
                baurocs.append(BinaryAUROC()(1-s[:, 0], t).item())
                bauprcs.append(AveragePrecision(task="binary")(1-s[:, 0], t.type(torch.LongTensor)).item())
                
                bin_prec = torch.sum(torch.logical_and(pred!=0, y!=0).type(torch.float32))/torch.sum((pred!=0).type(torch.float32))
                bin_rec = torch.sum(torch.logical_and(pred!=0, y!=0).type(torch.float32))/torch.sum((y!=0).type(torch.float32))
                bin_f1 = (2*bin_prec*bin_rec)/(bin_prec+bin_rec)
                bf1s.append(bin_f1.item())
                bin_acc = torch.sum(torch.logical_or(torch.logical_and(pred==0, y==0).type(torch.float32), torch.logical_and(pred!=0, y!=0).type(torch.float32)))/len(y)
                baccs.append(bin_acc.item())
                brecalls.append(bin_rec.item())



                inp = torch.Tensor(X_train).to('cpu')
                if(dtu == 'mae_full' and mt != 'GRU'):
                    inp = inp.swapaxes(1, 2)
                y = torch.Tensor(y_mc_train).to('cpu')

                pred = model(inp)#.reshape(y.shape)
                pred = torch.argmax(pred, axis=1)
               
                print("\nTrain MSE:", nn.MSELoss()(pred, y))

                acc = torch.mean((pred==y).type(torch.float32))
                
                print("Train ACC:", acc)
                macro = 0
                for b in range(buckets+1):
                    precision = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((pred==b).type(torch.float32))
                    recall = torch.sum(torch.logical_and(pred==b, y==b).type(torch.float32))/torch.sum((y==b).type(torch.float32))
                    f1 = (2*precision*recall)/(precision+recall)
                    macro+=recall
                print("Macro: " + str(macro/(buckets+1)))
                print("\n\n")
                
print('t1accs', np.mean(t1accs), 2*np.std(t1accs))
print('f1s', np.mean(f1s), 2*np.std(f1s))
print('aurocs', np.mean(aurocs), 2*np.std(aurocs))
print('mses', np.mean(mses), 2*np.std(mses))
# print(np.mean(mmses), 2*np.std(mmses))
print()
print('baccs', np.mean(baccs), 2*np.std(baccs))
print('bf1s', np.mean(bf1s), 2*np.std(bf1s))
print('baurocs', np.mean(baurocs), 2*np.std(baurocs))
print('bauprcs', np.mean(bauprcs), 2*np.std(bauprcs))
print('brecalls', np.mean(brecalls), 2*np.std(brecalls))



torch.Size([72, 768]) torch.Size([72])
0.05125931277871132
Test MSE: tensor(0.4800)
Test ACC: tensor(0.5200)

Class: 0 Precision: 0.8977272510528564 Recall: 0.5129870176315308 F1: 0.652892529964447 AUROC: 0.6137909889221191 MSE: 0.48701298236846924
Class: 1 Precision: 0.13793103396892548 Recall: 0.5714285969734192 F1: 0.2222222238779068 AUROC: 0.6137909889221191 MSE: 0.4285714328289032
Metrics of Interest (MSE, f1, auroc, mse, Bin AUROC): 
Test MSE: 0.47999998927116394 F1: 0.4375573694705963, AUROC: 0.6137909889221191, MSE: 0.4577922224998474, Bin AUROC: 0.6137909889221191


Train MSE: tensor(0.)
Train ACC: tensor(1.)
Macro: tensor(1.)



torch.Size([72, 768]) torch.Size([72])
0.05790957808494568
Test MSE: tensor(0.4914)
Test ACC: tensor(0.5086)

Class: 0 Precision: 0.9047619104385376 Recall: 0.4935064911842346 F1: 0.638655424118042 AUROC: 0.6014223098754883 MSE: 0.5064935088157654
Class: 1 Precision: 0.1428571492433548 Recall: 0.6190476417541504 F1: 0.2321428656578064 AUROC: 0.6014224

: 